In [ ]:
import pandas as pd
import datetime
import os
orig_dir = os.getcwd()
os.chdir("../")

In [4]:
from statbank import StatbankClient
os.chdir(orig_dir)

### Opprett client

In [5]:
client = StatbankClient(loaduser = "LAST360")

Lastepassord (TEST): ········


In [4]:
date = client.date_picker()

DatePicker(value=datetime.datetime(2022, 10, 25, 9, 54, 51, 501599), description='Publish-date', step=1)

In [5]:
client.set_publish_date(date)

Publishing date set to: 2022-10-26


### Hent filbeskrivelse

In [6]:
filbeskrivelse = client.get_description(tableid="06339")

Hentet uttaksbeskrivelsen for MedlemKristTrus, 
        med tabellid: 06339
        den 24.10.2022 klokka 09:55


In [7]:
print(filbeskrivelse)

Uttrekksbeskrivelse for statbanktabell 06339. 
        loaduser: LAST360. 
        
        Hele filbeskrivelsen "rå" ligger under .filbeskrivelse
        Andre attributter: 
        .deltabelltitler, .kodelister, .prikking, .variabler

Deltabell (DataFrame) nummer 1: medlemkristtrus1.dat
Antall kolonner: 3
	Kolonne 1: Kristne trussamfunn
	Kolonne 2: Tidsperioden for tabelldataene, enhet = år, format = åååå
	Kolonne 3: Medlemmer i kristne trus- og livssynssamfunn
Eksempellinje: 999;2020;3
        


In [8]:
filbeskrivelser = client.get_description_batch(tableids=["06339", "05300"])

Hentet uttaksbeskrivelsen for MedlemKristTrus, 
        med tabellid: 06339
        den 24.10.2022 klokka 09:58
Hentet uttaksbeskrivelsen for Kulturbaromet58, 
        med tabellid: 05300
        den 24.10.2022 klokka 09:58


In [9]:
print(filbeskrivelser["05300"])

Uttrekksbeskrivelse for statbanktabell 05300. 
        loaduser: LAST360. 
        
        Hele filbeskrivelsen "rå" ligger under .filbeskrivelse
        Andre attributter: 
        .deltabelltitler, .kodelister, .prikking, .variabler

Deltabell (DataFrame) nummer 1: kulturbaromet581.dat
Antall kolonner: 5
	Kolonne 1: Avstand til kulturtilbud
	Kolonne 2: Ulike kulturtilbud
	Kolonne 3: Tidsperioden for tabelldataene, enhet = år, format = åååå
	Kolonne 4: Avstand til nærmeste lokale/sted
	Kolonne 5: Prikking for kolonne 4: Avstand
Eksempellinje: 06;08;2021;4;
        


### Konstruer data

In [10]:
dfs_06339 = pd.read_parquet("06339_statbank.parquet.gzip")
type(dfs_06339)

pandas.core.frame.DataFrame

In [11]:
dfs_05300 = pd.read_parquet("05300_statbank.parquet.gzip")
type(dfs_05300)

pandas.core.frame.DataFrame

### Valider data

In [12]:
client.validate(dfs_06339, "06339")

Hentet uttaksbeskrivelsen for MedlemKristTrus, 
        med tabellid: 06339
        den 24.10.2022 klokka 09:58

validating...
Correct number of DataFrames passed.
Correct number of columns...
No codes in categorical columns outside codelist.
No codes missing from categorical columns.
Timeformat validation ok.
Prikking-codes validation ok / No prikke-columns in use.



In [13]:
client.validate_batch({
    "06339": dfs_06339,
    "05300": dfs_05300, 
})

Hentet uttaksbeskrivelsen for MedlemKristTrus, 
        med tabellid: 06339
        den 24.10.2022 klokka 09:58

validating...
Correct number of DataFrames passed.
Correct number of columns...
No codes in categorical columns outside codelist.
No codes missing from categorical columns.
Timeformat validation ok.
Prikking-codes validation ok / No prikke-columns in use.

Hentet uttaksbeskrivelsen for Kulturbaromet58, 
        med tabellid: 05300
        den 24.10.2022 klokka 09:58

validating...
Correct number of DataFrames passed.
Correct number of columns...
No codes in categorical columns outside codelist.
Category codes missing from data (This is ok, just make sure missing data is intentional):
Code 02 missing from column number 1, in deltabell number 1, (kulturbaromet581.dat)
Code 03 missing from column number 1, in deltabell number 1, (kulturbaromet581.dat)
Code 08 missing from column number 2, in deltabell number 1, (kulturbaromet581.dat)

Timeformat validation ok.
Prikking-codes va

In [14]:
# Forbedringsforslag å slå opp disse?
for koder in filbeskrivelser["05300"].kodelister:
    print(koder['kodeliste'])
    for kode in koder['koder']:
        print(kode["kode"], kode["text"])
    print()

Avstand
01 Under 1 km
02 1-4,9 km
03 5-9,9 km
04 10-24 km
05 25-49 km
06 50 km eller over

UlikeKulturtilbud
01 Kino/lokale som brukes til kinoforestillinger
02 Teater/lokale som brukes til teaterforestillinger
03 Konsertsal/lokale som brukes til konserter
04 Galleri/kunstsamling
05 Museum
06 Idrettsplass/idrettshall
07 Bibliotek/offentlig boksamling
08 Bokhandel



### Transfer data

In [15]:
transfer = client.transfer(dfs_06339, "06339")

Hentet uttaksbeskrivelsen for MedlemKristTrus, 
        med tabellid: 06339
        den 24.10.2022 klokka 09:59

validating...
Correct number of DataFrames passed.
Correct number of columns...
No codes in categorical columns outside codelist.
No codes missing from categorical columns.
Timeformat validation ok.
Prikking-codes validation ok / No prikke-columns in use.

<Response [200]>
Publisering satt til: 2022-10-26 08:00
Følg med på lasteloggen (tar noen minutter): https://i.test.ssb.no/lastelogg/gui/197921
Og evt APIen?: https://i.test.ssb.no/lastelogg/api/197921


In [16]:
print(transfer)

Overføring for statbanktabell 06339. 
    loaduser: LAST360.
    Publisering: 2022-10-26.
    Lastelogg: https://i.test.ssb.no/lastelogg/gui/197921


In [27]:
# Headers skal være slettet fra transfer-objektet, selv om den ligger i clienten.
#transfer.response.request.__dict__

In [28]:
# Vent litt, ellers kræsjer den med forrige overføring
transfer_batch = client.transfer_batch({
    "06339": dfs_06339,
    "05300": dfs_05300,
})

Hentet uttaksbeskrivelsen for MedlemKristTrus, 
        med tabellid: 06339
        den 24.10.2022 klokka 10:01

validating...
Correct number of DataFrames passed.
Correct number of columns...
No codes in categorical columns outside codelist.
No codes missing from categorical columns.
Timeformat validation ok.
Prikking-codes validation ok / No prikke-columns in use.

<Response [200]>
Publisering satt til: 2022-10-26 08:00
Følg med på lasteloggen (tar noen minutter): https://i.test.ssb.no/lastelogg/gui/197922
Og evt APIen?: https://i.test.ssb.no/lastelogg/api/197922
Hentet uttaksbeskrivelsen for Kulturbaromet58, 
        med tabellid: 05300
        den 24.10.2022 klokka 10:01

validating...
Correct number of DataFrames passed.
Correct number of columns...
No codes in categorical columns outside codelist.
Category codes missing from data (This is ok, just make sure missing data is intentional):
Code 02 missing from column number 1, in deltabell number 1, (kulturbaromet581.dat)
Code 03 mi

In [29]:
for transfer in transfer_batch.values():
    print(repr(transfer))

StatbankTransfer([data], tabellid="06339", loaduser="LAST360")
StatbankTransfer([data], tabellid="05300", loaduser="LAST360")


### Logging

In [30]:
print(client)

StatbankClient for user LAST360
        Publishing at 2022-10-26
        Shortuser cfc
        Sending mail to cfc
        And sending mail to cfc
        Overwrite set to True
        Approve set to 2
        Validation set to True
        
        Log:
        Datepicker created at 2022-10-24 09:55
	Date set to 2022-10-26 at 2022-10-24 09:55
	Getting description for tableid 06339 at 2022-10-24 09:55
	Got description for tableid 06339 at 2022-10-24 09:58
	Got description for tableid 05300 at 2022-10-24 09:58
	Validated data for tableid 06339 at 2022-10-24 09:58
	Validated data for tableid 06339 at 2022-10-24 09:58
	Validated data for tableid 05300 at 2022-10-24 09:58
	Transferring tableid 06339 at 2022-10-24 09:59
	Transferred tableid 06339 at 2022-10-24 10:01
	Transferred tableid 05300 at 2022-10-24 10:01


In [31]:
client.log

['Datepicker created at 2022-10-24 09:55',
 'Date set to 2022-10-26 at 2022-10-24 09:55',
 'Getting description for tableid 06339 at 2022-10-24 09:55',
 'Got description for tableid 06339 at 2022-10-24 09:58',
 'Got description for tableid 05300 at 2022-10-24 09:58',
 'Validated data for tableid 06339 at 2022-10-24 09:58',
 'Validated data for tableid 06339 at 2022-10-24 09:58',
 'Validated data for tableid 05300 at 2022-10-24 09:58',
 'Transferring tableid 06339 at 2022-10-24 09:59',
 'Transferred tableid 06339 at 2022-10-24 10:01',
 'Transferred tableid 05300 at 2022-10-24 10:01']

### Apidata kan hentes fra under clienten

In [6]:
client.apidata_all("05300")

https://data.ssb.no/api/v0/no/table/05300/


,avstand,kulturtilbud,statistikkvariabel,år,value
0,Under 1 km,Kino eller lokale med jevnlig spillefilmframvi...,Avstand til nærmeste lokale/sted,1991,16
1,Under 1 km,Kino eller lokale med jevnlig spillefilmframvi...,Avstand til nærmeste lokale/sted,1994,14
2,Under 1 km,Kino eller lokale med jevnlig spillefilmframvi...,Avstand til nærmeste lokale/sted,1997,12
3,Under 1 km,Kino eller lokale med jevnlig spillefilmframvi...,Avstand til nærmeste lokale/sted,2000,12
4,Under 1 km,Kino eller lokale med jevnlig spillefilmframvi...,Avstand til nærmeste lokale/sted,2004,11
...,...,...,...,...,...
427,50 km eller over,Bokhandel,Avstand til nærmeste lokale/sted,2004,3
428,50 km eller over,Bokhandel,Avstand til nærmeste lokale/sted,2008,3
429,50 km eller over,Bokhandel,Avstand til nærmeste lokale/sted,2012,3
430,50 km eller over,Bokhandel,Avstand til nærmeste lokale/sted,2016,2


#### Men kan også importeres seperat, krever da ikke passord

In [8]:
from statbank.apidata import apidata_all

In [9]:
apidata_all("05300")

https://data.ssb.no/api/v0/no/table/05300/


,avstand,kulturtilbud,statistikkvariabel,år,value
0,Under 1 km,Kino eller lokale med jevnlig spillefilmframvi...,Avstand til nærmeste lokale/sted,1991,16
1,Under 1 km,Kino eller lokale med jevnlig spillefilmframvi...,Avstand til nærmeste lokale/sted,1994,14
2,Under 1 km,Kino eller lokale med jevnlig spillefilmframvi...,Avstand til nærmeste lokale/sted,1997,12
3,Under 1 km,Kino eller lokale med jevnlig spillefilmframvi...,Avstand til nærmeste lokale/sted,2000,12
4,Under 1 km,Kino eller lokale med jevnlig spillefilmframvi...,Avstand til nærmeste lokale/sted,2004,11
...,...,...,...,...,...
427,50 km eller over,Bokhandel,Avstand til nærmeste lokale/sted,2004,3
428,50 km eller over,Bokhandel,Avstand til nærmeste lokale/sted,2008,3
429,50 km eller over,Bokhandel,Avstand til nærmeste lokale/sted,2012,3
430,50 km eller over,Bokhandel,Avstand til nærmeste lokale/sted,2016,2
